In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH TEMP0 AS (
    SELECT store_id, MIN(calday) cutoff_date FROM (
      SELECT 
        cast(store_id as STRING), 
        DATE_ADD(calday, -1) AS calday 
      FROM {catalog_name}.udp_wcm_silver_rop.fms_skustock
      WHERE store_id RLIKE 'F[0-9]' GROUP BY 1, 2 HAVING SUM(IFNULL(CAST(stock_qty AS FLOAT), 0)) != 0
      UNION ALL 
      SELECT cast(store_id as STRING), calday FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_tranline
      WHERE store_id RLIKE 'F[0-9]' GROUP BY 1, 2 HAVING SUM(IFNULL(sale_qty,0)) != 0
    ) A GROUP BY 1
  )
SELECT  A.*, 
  IFNULL(B.ref_fc_store, A.store_id) ref_fc_store, 
  B.supply_store, 
  IFNULL(B.addr_numbr, A.store_id) addr_numbr 
FROM TEMP0 A LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store B 
  ON try_cast(A.store_id as STRING) = try_cast(B.store_id as STRING)
  """)